This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [ ]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [ ]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [ ]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
api = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
payload = {'start_date':'2020-11-30', 'api_key': API_KEY}
res = requests.get(api,params=payload)

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(res.json())

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

The following code imports the date as a JSON file for AFX_X stock for 2017.

In [ ]:
# 1 Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
afx_x = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
payload = {'start_date':'2017-01-01','end_date':'2017-12-31', 'api_key': API_KEY}
res = requests.get(afx_x,params=payload)


The following code saves the previously aquired data as a python dictionary and the data as a 2-D list, with each row identifying a date, and each value within a row is indexed as followed:
0.  'Date'
1.  'Open'
2.  'High'
3.  'Low'
4.  'Close'
5.  'Change'
6.  'Traded Volume'
7.  'Turnover'
8.  'Last Price of the Day'
9.  'Daily Traded Units'
10. 'Daily Turnover'

In [ ]:
# 2 Convert the returned JSON object into a Python dictionary.
json_dict = res.json()['dataset_data']
col = json_dict['column_names']
json_data = json_dict['data']
print(col)

The following code calculates the highest and lowest opening prices were for the stock in this period. None values were discarded.

In [ ]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.
opening = [item[1] for item in json_data]
opening = [i for i in opening if i is not None]
highest = max(opening)
lowest = min(opening)
print('The highest opening price is: €' + str(round(highest,2)))
print('The lowest opening price is: €' + str(round(lowest,2)))

As you can see, the highest opening price was €53.11 and the lowest was €34.00.

In [ ]:
# 4. What was the largest change in any one day (based on High and Low price)?
high = [item[2] for item in json_data]
low = [item[3] for item in json_data]
change = []
for i, hv in enumerate(high):
    lv = low[i]
    if lv is not None and hv is not None:
        change.append(hv-lv)

max_change = max(change)

print('The largest change in price in one day is: €' + str(round(max_change,2)))


The largest change in price in one day is €2.81.

In [ ]:
# 5. What was the largest change between any two days (based on Closing Price)?
largest_change = -float('inf')
closing = [item[4] for item in json_data]
prior = closing[0]
length = len(closing)

for i in range(1,length):
    curr = closing[i]
    if curr is not None and prior is not None:
        change = abs(curr-prior)
        largest_change = change if change > largest_change else largest_change
    prior = curr
print('The largest change in price in a two day period is: €' + str(round(largest_change,2)))

The largest change in price in a two day period is €2.56

In [ ]:
# 6. What was the average daily trading volume during this year?
trading = [item[6] for item in json_data]
trading = [i for i in trading if i is not None]
avg_trading = sum(trading)/len(trading)
print('The average daily trading volume this year was ' + str(round(avg_trading)) + ' stocks.')

The average daily trading volume this year(2017) was 89124 stocks.

In [ ]:
# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
trading = [item[6] for item in json_data]
trading = [i for i in trading if i is not None]
trading.sort()
length = len(trading)
median = trading[int(length/2)] if length % 2 == 1 else (trading[int(length/2)] + trading[int(length/2) - 1])/2

print('The median trading volume during this year per day was ' + str(round(median)) + ' stocks.')

The median trading volume during this year per day was 76286 stocks.